In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/spaceship-titanic/train.csv').drop(['PassengerId', 'Name', 'HomePlanet'], axis=1)
test = pd.read_csv('../input/spaceship-titanic/test.csv')
Passenger_Id = test.copy().PassengerId.values
test = test.drop(['PassengerId', 'Name', 'HomePlanet'], axis=1)

combined = [data, test]


In [ ]:
print(data.shape)
#print(data.dropna().shape)
combined[0].info()

In [ ]:
cols = ['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
cols_combined = ['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService']
#for i in combined:
#    a = i['RoomService'].values
#    
#    for col in cols:
#        a += i[col].values
#    
#    i.insert(10, 'TotalSpent', a)


a = combined[0]['RoomService'].values
b = combined[0]['ShoppingMall'].values
c = combined[0]['FoodCourt'].values
d = combined[0]['Spa'].values
e = combined[0]['VRDeck'].values
combined[0].insert(10, 'TotalSpent', a+b+c+d+e)

a = combined[1]['RoomService'].values
b = combined[1]['ShoppingMall'].values
c = combined[1]['FoodCourt'].values
d = combined[1]['Spa'].values
e = combined[1]['VRDeck'].values
combined[1].insert(10, 'TotalSpent', a+b+c+d+e)


In [ ]:
a = combined[0]['Cabin'].str.split('/', expand=True)
b = combined[1]['Cabin'].str.split('/', expand=True)
combined[0]['Cabin'] = combined[0]['Cabin'].str.split('/', expand=True)[0].values
combined[1]['Cabin'] = combined[1]['Cabin'].str.split('/', expand=True)[0].values

combined[0].insert(9, 'CabinSize', a[2])
combined[1].insert(9, 'CabinSize', b[2])

combined[0].insert(10, 'CabinNum', a[1])
combined[1].insert(10, 'CabinNum', b[1])


In [ ]:
import seaborn as sns
from sklearn import preprocessing

cols_to_process = ['VIP', 'CryoSleep', 'Destination', 'Cabin', 'CabinSize', 'CabinNum']
le = preprocessing.LabelEncoder()
for i in combined:
    for col in cols_to_process:
        i[col] = le.fit_transform(i[col])

In [ ]:
for i in combined:
    for j in i:
        name = str(j)
        if name == 'Cabin':
            i[j].fillna(7, inplace=True)
            continue
        elif name == 'CabinSize':
            i[j].fillna(7, inplace=True)
        else:
            i[j].fillna(i[j].median(), inplace=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(combined[0].drop(['Transported'], axis=1), combined[0]['Transported'].values)

In [ ]:
log_model = LogisticRegression(max_iter=10000).fit(train_x, train_y)
log_pred = log_model.predict(val_x)
log_acc = accuracy_score(val_y, log_pred)

In [ ]:
from sklearn.naive_bayes import GaussianNB
bayes_model = GaussianNB().fit(train_x, train_y)
naive_pred = bayes_model.predict(val_x)
naive_acc = accuracy_score(val_y, naive_pred)

In [ ]:
from sklearn.neural_network import MLPClassifier
classifier_model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(6,2)).fit(train_x, train_y)
nn_pred = classifier_model.predict(val_x)
nn_acc = accuracy_score(val_y, nn_pred)

In [ ]:
from sklearn import svm
svm_model = svm.SVC().fit(train_x, train_y)
svm_pred = svm_model.predict(val_x)
svm_acc = accuracy_score(val_y, svm_pred)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier().fit(train_x, train_y)
knn_pred = knn_model.predict(val_x)
knn_acc = accuracy_score(val_y, knn_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(random_state=0, n_estimators=475, class_weight='balanced').fit(train_x, train_y)
forest_pred = forest_model.predict(val_x)
forest_acc = accuracy_score(val_y, forest_pred)

In [ ]:
from xgboost import XGBClassifier
XGB_model = XGBClassifier(n_estimators=230, seed=0, scale_pos_weight=1.5).fit(train_x, train_y)
XGB_pred = XGB_model.predict(val_x)
XGB_acc = accuracy_score(val_y, XGB_pred)

In [ ]:
print('logicistic: ', log_acc)
print('naive_bayes: ', naive_acc)
print('nn: ', nn_acc)
print('svm: ', svm_acc)
print('knn: ', knn_acc)
print('forest: ', forest_acc)
print('XGB: ', XGB_acc)

In [ ]:
#grab all the prediction put them in
log_model = XGBClassifier(n_estimators=230, seed=0, scale_pos_weight=1.5).fit(combined[0].drop('Transported',axis=1), combined[0]['Transported'].values)
final_pred = log_model.predict(combined[1])
submission_csv = pd.DataFrame({'PassengerId': Passenger_Id, 'Transported': final_pred})
submission_csv.to_csv('submission.csv', index=False)

In [ ]:
combined[0].head(10)